In [1]:
import pandas as pd

In [30]:
fund_navs = {
    'USCB': ['US CB Fund', 1128462384],
    'WWCB': ['Global CB Fund', 389452323],
    'MSFD': ['Multi-strat Fund', 2879546658],
    'DDFD': ['Drawdown Fund 1', 200000000],
    'DFD2': ['Drawdown Fund 1', 104000000]
}

df_fund_navs = pd.DataFrame.from_dict(fund_navs, orient='index', columns=['fund_name', 'nav'])

df_fund_navs['nav_%_firm'] = df_fund_navs['nav'] / df_fund_navs['nav'].sum()
display(df_fund_navs)

,fund_name,nav,nav_%_firm
USCB,US CB Fund,1128462384,24.00%
WWCB,Global CB Fund,389452323,8.28%
MSFD,Multi-strat Fund,2879546658,61.25%
DDFD,Drawdown Fund 1,200000000,4.25%
DFD2,Drawdown Fund 1,104000000,2.21%


In [15]:
fund_pos = {
    'USCB_ABSD_4.25_1/26': ['USCB', 'ABSD_4.25_1/26', 'Convertible Bond', 106.25, 1000.00, 1000000.00, 1064093.75],
    'WWCB_ABSD_4.25_1/26': ['WWCB', 'ABSD_4.25_1/26', 'Convertible Bond', 106.25, 5000.00, 5000000.00, 5320468.75]
}

df_fund_pos = pd.DataFrame.from_dict(fund_pos, orient='index', columns=['fund_code', 'security_name', 'security_type', 'mark', 'quantity', 'notional', 'market_value'])



In [16]:
pd.options.display.float_format = '{:.02%}'.format

fund_pos_tgt= {
    'USCB_ABSD_4.25_1/26': [0.02],
    'WWCB_ABSD_4.25_1/26': [0.015],
    'MSFD_ABSD_4.25_1/26': [0.0075],
    'DDFD_ABSD_4.25_1/26': [0.03],
    'DFD2_ABSD_4.25_1/26': [0.05],    
}

df_fund_pos_tgt = pd.DataFrame.from_dict(fund_pos_tgt, orient='index', columns=['default_exposure_target_%_nav'])


In [23]:
df = df_fund_pos.join(df_fund_navs, on='fund_code', how='left').join(df_fund_pos_tgt, how='left')
display(df)

,fund_code,security_name,security_type,mark,quantity,notional,market_value,fund_name,nav,default_exposure_target_%_nav
USCB_ABSD_4.25_1/26,USCB,ABSD_4.25_1/26,Convertible Bond,10625.00%,100000.00%,100000000.00%,106409375.00%,US CB Fund,1128462384,2.00%
WWCB_ABSD_4.25_1/26,WWCB,ABSD_4.25_1/26,Convertible Bond,10625.00%,500000.00%,500000000.00%,532046875.00%,Global CB Fund,389452323,1.50%


In [27]:
df['exposure_%_nav'] = df['market_value'] / df['nav']

display(df[['security_name', 'exposure_%_nav', 'default_exposure_target_%_nav']])

,security_name,exposure_%_nav,default_exposure_target_%_nav
USCB_ABSD_4.25_1/26,ABSD_4.25_1/26,0.09%,2.00%
WWCB_ABSD_4.25_1/26,ABSD_4.25_1/26,1.37%,1.50%


In [92]:
trades = {
    1: ['ABSD_4.25_1/26', 'Convertible Bond', 105.375, 10000, 10000000, '10:15 AM']
}

df_trades = pd.DataFrame.from_dict(trades, orient='index', columns=['security_name', 'security_type', 'price', 'quantity', 'notional', 'timestamp'])

In [105]:
def allocate_trade_by_fund(trade):
    
    
    df_alloc = pd.DataFrame(df_fund_navs[['nav_%_firm']] * trade['notional'])
    
    df_alloc.index = df_alloc.index.values + '_'+ [trade['security_name']]
    
    return df_alloc
    
    

In [106]:
pd.options.display.float_format = '{:,.2f}'.format

df = allocate_trade_by_fund(df_trades.iloc[0,:])
display(df)

,nav_%_firm
USCB_ABSD_4.25_1/26,"2,400,237.49"
WWCB_ABSD_4.25_1/26,"828,364.40"
MSFD_ABSD_4.25_1/26,"6,124,790.64"
DDFD_ABSD_4.25_1/26,"425,399.65"
DFD2_ABSD_4.25_1/26,"221,207.82"


In [103]:
df_fund_pos

,fund_code,security_name,security_type,mark,quantity,notional,market_value
USCB_ABSD_4.25_1/26,USCB,ABSD_4.25_1/26,Convertible Bond,106.25,"1,000.00","1,000,000.00","1,064,093.75"
WWCB_ABSD_4.25_1/26,WWCB,ABSD_4.25_1/26,Convertible Bond,106.25,"5,000.00","5,000,000.00","5,320,468.75"
